In [ ]:
!pip install accelerate datasets

In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer,AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
math_dataset_arith = load_dataset('math_dataset','arithmetic__add_or_sub')
subset_dataset = math_dataset_arith["train"].select(range(100))

In [ ]:
class AgentManager:
    def __init__(self):
        self.agents = {}

    def create_agent(self, model_name, nick_name):
        config = BitsAndBytesConfig(load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name,config = config, device_map = "auto")
        self.agents[nick_name] = {'model': model, 'tokenizer': tokenizer}
        print(f"Agent '{nick_name}' created successfully.")

    def interact_with_agent(self, agent_name, prompt):
        # if agent_name not in self.agents:
        #     print(f"Agent '{agent_name}' not found.")
        #     return None
#         print("Agent present")
        agent = self.agents[agent_name]
        input_ids = agent['tokenizer'](prompt, return_tensors="pt").to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = agent['model'].generate(**input_ids, max_length=2048)
        # decode(outputs[0])
        return agent['tokenizer'].decode(outputs[0])

In [ ]:
from huggingface_hub import login
login()

In [ ]:
agent = AgentManager()
agent.create_agent("google/gemma-2b-it",'gemma_1')
agent.create_agent("google/gemma-2b-it",'gemma_2')

In [ ]:
math_dataset_arith["question"] = subset_dataset["question"][:50]
math_dataset_arith["answer"] = subset_dataset["answer"][:50]

In [ ]:
def summarize_responses_modified(response1, response2, q1):
    summarized_response = str(q1) + "  This is gemma agent's answer for the question: " + response1 + " This is another agent's answer for same question - " + response2 + " Now, Recheck those two answers and give the correct answer?."
    return summarized_response
import re

In [ ]:
def clean_response(response, question):
    """ Remove special tokens and unnecessary repetition of the question. """
    response = response.replace("<bos>", "").replace("<eos>", "").strip()
#     response = response.replace(question, "").strip()
    return response

rounds = 2
responses = []
ques = 0

for question in math_dataset_arith['question'][25:26]:
    question = question.split("'")[1].rstrip("\\n")
    current_input = question  # Initial input is just the question

    for i in range(rounds):
        # Interaction with Gemma 1
        response_gemma_1 = agent.interact_with_agent('gemma_1', current_input)
        response_gemma_1 = clean_response(response_gemma_1,question)
        print(f"For round {i} Gemma 1: answer = {response_gemma_1}, input = {current_input}")

        # Interaction with Gemma 2
        response_gemma_2 = agent.interact_with_agent('gemma_2', current_input)
        response_gemma_2 = clean_response(response_gemma_2,question)
        # print(f"For round {i} Gemma 2: answer = {response_gemma_2}, input = {current_input}")

        # Creating a summarized response without repeating the original question within responses
        summarized_response = f"For the question \"{question}\", Gemma 1's answer is {response_gemma_1} and Gemma 2's answer is {response_gemma_2} Recheck all the answers and see which is correct."
#         print(f"The round {i} summarized response is: {summarized_response}")
        # print("====="*50)

        # Update current_input to the latest summarized response for the next round
        current_input = summarized_response
    responses.append(summarized_response)
    print(f"{ques} done!")
    ques += 1

# print(responses)


In [ ]:
inputs = "Answer this question and give the final answer: For the question Put together -0.085 and 482., Gemma 1's answer is 5.Sure, here's the solution:-0.085 + 482.5 = 482.415 and Gemma 2's answer is 5.Sure, here's the solution:-0.085 + 482.5 = 482.415 Recheck all the answers and now check which is right by calculating again with their response and give the final answer."

In [ ]:
inputs_2 = agent.interact_with_agent('gemma_2', inputs)
print(inputs_2)

In [ ]:
def clean_response(response, question):
    """ Remove special tokens and unnecessary repetition of the question. """
    response = response.replace("<bos>", "").replace("<eos>", "").strip()
#     response = response.replace(question, "").strip()
    return response

In [ ]:
# cleaned_2 = clean_response(inputs_2,"Put together -0.085 and 482")
# cleaned_2 = cleaned_2 + "Recheck all the answers and now check which is right by calculating again with their response and give the final answer."
# trail = "Given the arithmetic problem of combining -0.085 and 482, we have two differing solutions. Gemma 1 claims the result is 482.415, while Gemma 2 claims it is 5. Review these solutions, perform the necessary calculations, and advise which solution is correct, explaining your reasoning."
# trail_2 = trail + "Based on the previous discussion, please re-evaluate the calculations for both Gemma 1 and Gemma 2's claims, verify the accuracy of the conclusions drawn, and provide an in-depth analysis of any discrepancies. Ensure the arithmetic is clearly detailed and explain any errors in the initial assessments."
# inputs = "Previous Discussions: Put together -0.085 and 482., Gemma 1's answer is 5.Sure, here's the solution:-0.085 + 482.5 = 482.415 and Gemma 2's answer is 5.Sure, here's the solution:-0.085 + 482.5 = 482.415 "

# add_this = "Based on the previous discussion, please re-evaluate the calculations for both Gemma 1 and Gemma 2's claims, verify the accuracy of the conclusions drawn, and provide an in-depth analysis of any discrepancies.Provide one correct final answer according to you. Mention the value of the final answer. Ensure the arithmetic is clearly detailed and explain any errors in the initial assessments."

## Summarisation + Cross Feeding

In [ ]:
def clean_response(response):
    """ Remove special tokens and unnecessary repetition of the question. """
    response = response.replace("<bos>", "").replace("<eos>", "").strip()
#     response = response.replace(question, "").strip()
    return response

rounds = 2
responses = []
ques = 0

for question in math_dataset_arith['question'][48:60]:
    question = question.split("'")[1].rstrip("\\n")
    current_input = question  # Initial input is just the question

    for i in range(rounds):
        if i == 0:
            # Interaction with Gemma 1
            response_gemma_1 = agent.interact_with_agent('gemma_1', current_input)
            response_gemma_1 = clean_response(response_gemma_1)
#             responses.append(response_gemma_1)
#             print(f"For round {i} Gemma 1: answer = {response_gemma_1}")

            # Interaction with Gemma 2
            response_gemma_2 = agent.interact_with_agent('gemma_2', current_input)
            response_gemma_2 = clean_response(response_gemma_2)
#             responses.append(response_gemma_2)
#             print(f"For round {i} Gemma 2: answer = {response_gemma_2}")

            # Creating a summarized response without repeating the original question within responses
            summarized_response = f"For the question \"{question}\", Gemma 1's answer is {response_gemma_1} and Gemma 2's answer is {response_gemma_2}."
            current_input = summarized_response
            print("======"*10)
        else:
#             prompt_1 = "Based on the previous discussion, please re-evaluate the calculations for both Gemma 1 and Gemma 2's claims, verify the accuracy of the conclusions drawn, and provide an in-depth analysis of any discrepancies.Provide one correct final answer according to you." 
            prompt_1 = "These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer."
            response_gemma_1 = agent.interact_with_agent('gemma_1', current_input + prompt_1)
            response_gemma_1 = clean_response(response_gemma_1)
#             responses.append(response_gemma_1)
#             print(f"For round {i} Gemma 1: answer = {response_gemma_1}")
            
#             cleaned = response_gemma_1.replace(prompt_1,"")
            prompt_2 = "Do you think the reevalution was correct?"
            response_gemma_2 = agent.interact_with_agent('gemma_2', response_gemma_1 + prompt_2)
            response_gemma_2 = clean_response(response_gemma_2)
#             responses.append(response_gemma_2)
#             print(f"For round {i} Gemma 2: answer = {response_gemma_2}")
            
            print("======"*10)
    
    final = response_gemma_2
        
        
    responses.append(final)
    print(f"{ques} done!")
    ques += 1

# print(responses)


In [ ]:
len(responses)

## Summarisation technique - Final Method

In [ ]:
def clean_response(response):
    """ Remove special tokens and unnecessary repetition of the question. """
    response = response.replace("<bos>", "").replace("<eos>", "").strip()
#     response = response.replace(question, "").strip()
    return response

rounds = 2
responses = []
ques = 0

for question in math_dataset_arith['question'][25:50]:
    question = question.split("'")[1].rstrip("\\n")
    current_input = question  # Initial input is just the question

    for i in range(rounds):
        if i == 0:
            # Interaction with Gemma 1
            response_gemma_1 = agent.interact_with_agent('gemma_1', current_input)
            response_gemma_1 = clean_response(response_gemma_1)
#             responses.append(response_gemma_1)
#             print(f"For round {i} Gemma 1: answer = {response_gemma_1}")

            # Interaction with Gemma 2
            response_gemma_2 = agent.interact_with_agent('gemma_2', current_input)
            response_gemma_2 = clean_response(response_gemma_2)
#             responses.append(response_gemma_2)
#             print(f"For round {i} Gemma 2: answer = {response_gemma_2}")

            # Creating a summarized response without repeating the original question within responses
            summarized_response = f"For the question \"{question}\", Gemma 1's answer is {response_gemma_1} and Gemma 2's answer is {response_gemma_2}."
            current_input = summarized_response
            print("======"*10)
        else:
#             prompt_1 = "Based on the previous discussion, please re-evaluate the calculations for both Gemma 1 and Gemma 2's claims, verify the accuracy of the conclusions drawn, and provide an in-depth analysis of any discrepancies.Provide one correct final answer according to you." 
            prompt_1 = "These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer."
            response_gemma_1 = agent.interact_with_agent('gemma_1', current_input + prompt_1)
            response_gemma_1 = clean_response(response_gemma_1)
#             responses.append(response_gemma_1)
#             print(f"For round {i} Gemma 1: answer = {response_gemma_1}")
            
#             cleaned = response_gemma_1.replace(prompt_1,"")
            prompt_2 = "These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer."
            response_gemma_2 = agent.interact_with_agent('gemma_2', current_input + prompt_2)
            response_gemma_2 = clean_response(response_gemma_2)
            
            summarized_response = f"For the question \"{question}\", Gemma 1's answer is {response_gemma_1} and Gemma 2's answer is {response_gemma_2}."
#             responses.append(response_gemma_2)
#             print(f"For round {i} Gemma 2: answer = {response_gemma_2}")
            
            print("======"*10)
    
    final = summarized_response
        
        
    responses.append(final)
    print(f"{ques} done!")
    ques += 1

# print(responses)


In [ ]:
len(responses)

In [ ]:
import os

In [ ]:
directory = os.path.dirname('/kaggle/working/responses_gemma_second_batch_diff_prompt_2.txt')
if not os.path.exists(directory):
    os.makedirs(directory)

with open('/kaggle/working/responses_gemma_second_batch_diff_prompt_2.txt', "w") as file:
    # Write each element of the list to the file
    for item in responses:
        file.write(item + "\n")

print("List saved to", directory)

In [ ]:
len(responses)